In [2]:
import numpy as np

def generate_circle_points_in_high_dim(n_points=10, n_dims=128):
    """
    Generate n_points unit-norm points that lie on a 2D circle embedded in n_dims dimensions.
    Returns both the points and their singular values.
    """
    # Generate angles for the circle
    thetas = np.linspace(0, 2*np.pi, n_points, endpoint=False)
    
    # Create points on the unit circle in 2D
    points_2d = np.column_stack([np.cos(thetas), np.sin(thetas)])
    
    # Pad with zeros to get to n_dims dimensions
    points = np.pad(points_2d, ((0, 0), (0, n_dims-2)), mode='constant')
    
    # Stack points as rows in a matrix
    X = points
    
    # Compute singular values
    singular_values = np.linalg.svd(X, compute_uv=False)
    
    return X, singular_values

# Generate points and compute singular values
n_points = 100
n_dims = 128
X, singular_values = generate_circle_points_in_high_dim(n_points, n_dims)

# Verify properties
print(f"Shape of point matrix: {X.shape}")
print(f"\nFirst few points norms (should be 1.0):")
print(np.linalg.norm(X, axis=1)[:5])

print(f"\nFirst few singular values (should only have 2 non-zero values):")
print(singular_values[:5])

print(f"\nNumber of singular values > 1e-10: {np.sum(singular_values > 1e-10)}")

Shape of point matrix: (100, 128)

First few points norms (should be 1.0):
[1. 1. 1. 1. 1.]

First few singular values (should only have 2 non-zero values):
[7.07106781 7.07106781 0.         0.         0.        ]

Number of singular values > 1e-10: 2


In [35]:
def pca(t:np.ndarray):
    n, p = t.shape # expect 2d tensor
    centered = t - t.mean(axis=0)[None, :]
    # centered_normed = centered / np.linalg.norm(centered, axis=1)[:, None]
    # print("Norms of centered data:")
    # print(np.linalg.norm(centered , axis=1)[:5])
    u, s, vh = np.linalg.svd(centered)
    eig = s**2
    eig_sorted, ids = np.sort(eig)[::-1], np.argsort(eig)[::-1]
    eigenvectors = vh[ids, :]
    # t.T @ t = v s u.T u s vh = v s^2 vh; 
    return eigenvectors, eig_sorted

In [36]:
import numpy as np
from scipy import linalg

# Number of points to generate
n_points = 1024
n_dims = 128

# Generate angles for the circle
thetas = np.linspace(0, 2*np.pi, n_points, endpoint=False)
thetas_2 = np.random.rand(n_points) * 2 * np.pi

# Initialize points array
points = np.zeros((n_points, n_dims))

# Set the main circle components (using first 2 dimensions)
circle_radius = 0.1  # Main circle radius < 1 to leave room for other dimensions
points[:, 0] = circle_radius * np.cos(thetas)
points[:, 1] = circle_radius * np.sin(thetas)
circle_radius_2 = 0.1  # Main circle radius < 1 to leave room for other dimensions
points[:, 2] = circle_radius_2 * np.cos(thetas_2)
points[:, 3] = circle_radius_2 * np.sin(thetas_2)

# Fill remaining dimensions with a fixed pattern that preserves unit norm
# We'll use a small sine wave with decreasing amplitude
remaining_norm = np.sqrt(1 - circle_radius**2- circle_radius_2**2)  # This ensures unit norm
remaining_points = np.zeros((n_points, n_dims-4))
fill_with = np.random.randn(1, n_dims-4)
fill_with = fill_with / np.linalg.norm(fill_with) * remaining_norm
remaining_points[:, :] = fill_with
points[:, 4:] = remaining_points
# for i in range(2, n_dims):
#     # Decreasing amplitude for higher dimensions
#     amplitude = remaining_norm / np.sqrt(n_dims-2) 
#     points[:, i] = amplitude * np.sin(thetas * (i-1)/10)

# Verify unit norm
norms = np.linalg.norm(points, axis=1)
print("Point norms (should all be ≈1):")
print(norms[:5])
print(f"Mean norm: {np.mean(norms):.6f}")
print(f"Std norm: {np.std(norms):.6f}")

# Compute singular values
_, eigenvalues = pca(points)
from sklearn.decomposition import PCA
pca_sklearn = PCA(n_components=n_dims)
pca_sklearn.fit(points)
eigenvalues_sklearn = pca_sklearn.singular_values_**2
s = linalg.svd(points, compute_uv=False)
print("\nTop 10 singular values:")
print(s[:10]) 
print("\nTop 10 singular values squared:")
print(s[:10]**2)
print("\nTop 10 eigenvalues:")
print(eigenvalues[:10])
print("\nTop 10 eigenvalues sklearn:")
print(eigenvalues_sklearn[:10])

print("\nSum of singular values squared:")
print(np.sum(s**2))
print("\nSum of eigenvalues:")
print(np.sum(eigenvalues))
print("\nSum of eigenvalues sklearn:")
print(np.sum(eigenvalues_sklearn))

# # After running PCA:
# print("\nFirst 5 eigenvalues:")
# print("Our PCA:", eigenvalues[:5])
# print("Sklearn:", pca_sklearn.singular_values_[:5]**2)

# print("\nSums:")
# print("Our PCA:", np.sum(eigenvalues))
# print("Sklearn:", np.sum(pca_sklearn.singular_values_**2))

# # Verify that points lie on a circle in first 2 dimensions
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 10))
# plt.scatter(points[:, 0], points[:, 1])
# plt.axis('equal')
# plt.title('Projection onto first 2 dimensions')
# plt.show()

Point norms (should all be ≈1):
[1. 1. 1. 1. 1.]
Mean norm: 1.000000
Std norm: 0.000000

Top 10 singular values:
[3.16784159e+01 2.36300703e+00 2.26423070e+00 2.22000131e+00
 2.19977680e+00 7.87289989e-15 2.18698850e-15 9.58332402e-16
 8.79362373e-16 6.73553479e-16]

Top 10 singular values squared:
[1.00352203e+03 5.58380220e+00 5.12674067e+00 4.92840580e+00
 4.83901797e+00 6.19825527e-29 4.78291869e-30 9.18400993e-31
 7.73278182e-31 4.53674288e-31]

Top 10 eigenvalues:
[5.58380631e+00 5.12674082e+00 4.92840990e+00 4.83902006e+00
 1.55296838e-25 5.22981347e-32 5.22981347e-32 5.22981347e-32
 5.22981347e-32 5.22981347e-32]

Top 10 eigenvalues sklearn:
[5.58380631e+00 5.12674082e+00 4.92840990e+00 4.83902006e+00
 1.55296838e-25 5.22981347e-32 5.22981347e-32 5.22981347e-32
 5.22981347e-32 5.22981347e-32]

Sum of singular values squared:
1024.0000000000016

Sum of eigenvalues:
20.477977082163292

Sum of eigenvalues sklearn:
20.477977082163292


In [38]:
# Check norms before and after centering
print("Before centering:")
print("Mean norm:", np.mean(np.linalg.norm(points, axis=1)))

centered_points = points - points.mean(axis=0)[None, :]
print("\nAfter centering:")
print("Mean norm:", np.mean(np.linalg.norm(centered_points, axis=1)))

# Check total variance
print("\nTotal variance (sum of eigenvalues):", np.sum(eigenvalues))
print("Number of points:", n_points)
np.sum(np.sqrt(eigenvalues))


Before centering:
Mean norm: 1.0

After centering:
Mean norm: 0.14141261484549095

Total variance (sum of eigenvalues): 20.477977082163292
Number of points: 1024


9.047018132423418

In [43]:
import torch 

def debug_pca_comparison(t: torch.Tensor):
    # Your PCA
    u, s, vh = torch.linalg.svd((t - t.mean(dim=0, keepdim=True)))
    print("Direct SVD singular values (first 5):", s[:5])
    print("Direct SVD eigenvalues (first 5):", s[:5]**2)
    
    # Sklearn PCA
    pca_ = PCA(n_components=min(t.shape[1], t.shape[0]))
    t_np = t.cpu().numpy()
    pca_.fit(t_np)
    print("\nSklearn singular values (first 5):", pca_.singular_values_[:5])
    print("Sklearn eigenvalues * (n-1) (first 5):", pca_.explained_variance_[:5] * (t.shape[0]-1))



In [40]:
import torch
from sklearn.decomposition import PCA

def pca(t:torch.Tensor):
    n, p = t.shape # expect 2d tensor
    u, s, vh = torch.linalg.svd((t - t.mean(dim=0, keepdim=True)))
    eig = s**2
    eig_sorted, ids = torch.sort(eig, descending=True)
    eigenvectors = vh[ids, :]
    # t.T @ t = v s u.T u s vh = v s^2 vh; 
    return eigenvectors, eig_sorted

def sklearn_pca(t:torch.Tensor):
    pca_ = PCA(n_components=min(t.shape[1], t.shape[0]))
    t_np = t.cpu().numpy()
    pca_.fit(t_np)
    eigenvalues = pca_.explained_variance_
    return pca_.components_, eigenvalues


In [49]:
import torch

def generate_test_data(n_samples=1024, n_features=128, eps=1e-5):
    # Generate random data
    data = torch.randn(1, n_features)
    data = torch.randn(n_samples, n_features) * eps + data
    
    # Normalize each row to have unit norm
    data = data / torch.norm(data, dim=1, keepdim=True)
    
    # Verify normalization
    norms = torch.norm(data, dim=1)
    # print("Sample of row norms (should all be ≈1):")
    # print(norms[:5])
    # print(f"Mean norm: {torch.mean(norms):.6f}")
    # print(f"Std norm: {torch.std(norms):.6f}")
    
    return data

data = torch.randn(1024, 128)
torch_mean = data.mean(dim=0)
np_mean = data.cpu().numpy().mean(axis=0)
print(f"Mean difference: {torch.norm(torch_mean - np_mean)}")

for eps in [1e-7, 1e-5, 1e-3, 1e-1, 1]:
    # Generate data
    data = generate_test_data()
    # Run both PCA implementations
    eigenvectors, eigenvalues = pca(data)
    sklearn_eigenvectors, sklearn_eigenvalues = sklearn_pca(data)

    # Compare results
    print(f"\nEpsilon: {eps}")
    print("\nLargest eigenvalues from each method:")
    print("Our PCA:", eigenvalues[:5])
    print("Sklearn PCA (adjusted):", sklearn_eigenvalues[:5] * (data.shape[0]-1))

    torch_mean = data.mean(dim=0)
    np_mean = data.cpu().numpy().mean(axis=0)
    print(f"Mean difference: {torch.norm(torch_mean - np_mean)}")

# debug_pca_comparison(data)


Mean difference: 1.775522520119921e-07

Epsilon: 1e-07

Largest eigenvalues from each method:
Our PCA: tensor([1.6147e-09, 1.5705e-09, 1.5547e-09, 1.5402e-09, 1.5240e-09])
Sklearn PCA (adjusted): [2.4634737e-08 1.6103776e-09 1.5703415e-09 1.5492203e-09 1.5401717e-09]
Mean difference: 4.804496711585671e-06

Epsilon: 1e-05

Largest eigenvalues from each method:
Our PCA: tensor([1.4375e-09, 1.3823e-09, 1.3359e-09, 1.3139e-09, 1.3086e-09])
Sklearn PCA (adjusted): [2.9706820e-08 1.4359444e-09 1.3766718e-09 1.3307354e-09 1.3128650e-09]
Mean difference: 5.301290457282448e-06

Epsilon: 0.001

Largest eigenvalues from each method:
Our PCA: tensor([1.3439e-09, 1.2998e-09, 1.2844e-09, 1.2611e-09, 1.2519e-09])
Sklearn PCA (adjusted): [2.2825889e-08 1.3438364e-09 1.2966128e-09 1.2834197e-09 1.2607784e-09]
Mean difference: 4.65045559394639e-06

Epsilon: 0.1

Largest eigenvalues from each method:
Our PCA: tensor([1.3856e-09, 1.3781e-09, 1.3387e-09, 1.3168e-09, 1.2953e-09])
Sklearn PCA (adjusted): [2.

In [26]:
# Test implementation to compare methods
def compare_pca_methods(data):
    # Your implementation
    data_centered = data - data.mean(axis=0)[None, :]
    _, s1, _ = np.linalg.svd(data_centered)
    eig1 = s1**2
    
    # Sklearn's implementation
    pca_sklearn = PCA(n_components=data.shape[1])
    pca_sklearn.fit(data)
    eig2 = pca_sklearn.singular_values_**2
    
    print("First few eigenvalues from our implementation:", eig1[:5])
    print("First few eigenvalues from sklearn:", eig2[:5])
    print("Absolute difference in sums:", abs(np.sum(eig1) - np.sum(eig2)))
    print("Relative difference in first eigenvalue:", abs(eig1[0] - eig2[0])/eig1[0])
    
    # Also check the centered data is the same
    data_centered_sklearn = data - pca_sklearn.mean_[None, :]
    center_diff = np.max(np.abs(data_centered - data_centered_sklearn))
    print("Max difference in centered data:", center_diff)

compare_pca_methods(points)

First few eigenvalues from our implementation: [5.43382259e+00 5.12086314e+00 5.08466568e+00 4.83360798e+00
 1.52970985e-25]
First few eigenvalues from sklearn: [5.43382259e+00 5.12086314e+00 5.08466568e+00 4.83360798e+00
 1.52970985e-25]
Absolute difference in sums: 0.0
Relative difference in first eigenvalue: 0.0
Max difference in centered data: 0.0
